#このノートブックについて
このノートブックでは，Ollama と deepeval を使用して，大規模言語モデル (LLM) の評価を行います．Ollamaは，ローカル環境でLLMを実行するためのツールであり，deepevalは，LLMの評価のためのPythonライブラリです．

**"評価メトリックを定義する"**のcriteriaを変更することで，LLMの評価基準を変更することが出来ます．\
また，**"評価を実行して，tsvファイルに出力する．"**の*input, actual_output, expected_output* で入出力や教師文を設定することが出来ます．

---
このノートブックでは，以下の手順でLLMの評価を行います．

1. Google Driveをマウントする
2. 必要なライブラリをインストールする
3. Ollamaをインストールする
4. CUDAドライバーをインストールする
5. 環境変数を設定する
6. Ollamaサーバーをバックグラウンドで起動する
7. llama3.2モデルをダウンロードする
8. OllamaのPythonライブラリをインストールする
9. deepevalにローカルモデルを設定する
10. 評価メトリックを定義する
11. 評価を実行する
---
※私のデータセットで試したときは，T4で動きました（ハイメモリでなくても）．\
※途中，「セッションを再起動しなさい」という警告が出る場合があります．再起動するのが無難ですが，おそらく無視（キャンセル）でも大丈夫なはずです．

In [ ]:
#@title Google Driveをマウントする
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
#@title 必要なライブラリをインストールする
!pip install deepeval  # deepeval: LLMの評価のためのライブラリ
!pip install ngrok  # ngrok: ローカルサーバーを外部に公開するためのツール
!pip install pyngrok # pyngrok: ngrokをPythonから操作するためのライブラリ

In [ ]:
# Ollamaをインストールする
!curl https://ollama.ai/install.sh | sh

# CUDAドライバーをインストールする
!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers

# 環境変数を設定する
import os
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

In [ ]:
#@title Ollamaサーバーをバックグラウンドで起動する
!nohup ollama serve &

In [ ]:
#@title セッション終了時に，ollamaサーバを終了
import atexit
import os

def stop_ollama_server():
  os.system("pkill ollama")

atexit.register(stop_ollama_server)

In [ ]:
#@title llama3.2モデルをダウンロードする
# サーバーの起動を待つ
import time
time.sleep(10)

# llama3.2モデルをダウンロードする
!ollama pull llama3.2

In [ ]:
#@title OllamaのPythonライブラリをインストールする
!pip install ollama

In [ ]:
#@title deepevalにローカルモデルを設定する
import ollama
!deepeval set-local-model --model-name=llama3.2 \
    --base-url="http://localhost:11434/v1/" \
    --api-key="ollama"

In [ ]:
#@title タイムアウト時間を設定
import signal

class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException("60秒タイムアウトしました")

# タイムアウト時間を設定 (秒)
timeout_duration = 60

In [ ]:
#@title 評価メトリックを定義する
from deepeval.metrics import GEval # deepevalからGEvalメトリックをインポートします．GEvalは，生成されたテキストの正確性を評価するためのメトリックです．
from deepeval.test_case import LLMTestCaseParams # deepevalからLLMTestCaseParamsをインポートします．LLMTestCaseParamsは，テストケースのパラメータを定義するためのクラスです．
'''
correctness_metric = GEval(
    name="ここでメトリックの名前を設定．評価にはおそらく直接的には影響しない．",
    criteria="ここで評価基準を定義する",
    evaluation_params=[評価に使用するパラメータを設定．このサンプルでは，入力，実際の出力，期待される出力を使用する．],
    model = "llama3.2"  # 使用するモデルを"llama3
'''


correctness_metric = GEval(
    name="Correctness",  # メトリックの名前を"Correctness"に設定します．
    criteria="Determine whether the actual output is factually correct based on the expected output.",  #@param {type:"string"} メトリックの基準を設定します．この基準は，生成されたテキストが事実的に正しいかどうかを判断するために使用されます．
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT],  # 評価に使用するパラメータを設定します．ここでは，入力，実際の出力，期待される出力が使用されます．
    model = "llama3.2"  # 使用するモデルを"llama3
)

In [ ]:
#@title 評価を実行して，tsvファイルに出力する．
# 評価を実行する
import pandas as pd # pandasライブラリをインポートします．pandasは，データ分析に広く使用されるライブラリです．
from deepeval.test_case import LLMTestCase # deepevalライブラリからLLMTestCaseをインポートします．LLMTestCaseは，テストケースを表すクラスです．
from tqdm import tqdm  # tqdmをインポート
'''
    test_case = LLMTestCase(
        input=row['input_column'],  # 入力文 # テストケースを作成します．inputは，入力文をヘッダー名で指定します．
        actual_output=row['actual_output_column'],  # LLMの出力 # actual_outputは，LLMの実際の出力をヘッダー名で指定します．
        expected_output=row['expected_output_column']  # 正解 # expected_outputは，期待される出力をヘッダー名で指定します．
    )

    例（イメージ）：
    test_case = LLMTestCase(
        input="The dog chased the cat up the tree, who ran up the tree?",
        actual_output="It depends, some might consider the cat, while others might argue the dog.",
        expected_output="The cat."
    )
'''


# 入力ファイルと出力ファイルのパス
input_file = 'your_input_file(tsv)_path' # 入力ファイルのパスを指定します．入力ファイルは，タブ区切り値（TSV）形式である必要があります．
output_file = 'your_output_file(tsv)_path' # 出力ファイルのパスを指定します．出力ファイルは，タブ区切り値（TSV）形式で保存されます．

# TSVファイルを読み込み
data = pd.read_csv(input_file, sep='\t', nrows=None)   # nrowsは読み込み行数の制限

# スコアを計算して追加するためのリスト
scores = [] # スコアを格納するためのリストを初期化します．
reasons = [] # 理由を格納するためのリストを初期化します．


# 各行に対してスコアを計算
for _, row in tqdm(data.iterrows(), total=data.shape[0], desc="Evaluating"):
    try:
        # タイムアウトハンドラを設定
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(timeout_duration)  # アラームを設定

        test_case = LLMTestCase(
            input=row['input_column'],  #@param  # テストケースを作成します．inputは，入力文をヘッダー名で指定します．
            actual_output=row['actual_output_column'],  #@param {type:"string"} # LLMの出力 # actual_outputは，LLMの実際の出力をヘッダー名で指定します．
            expected_output=row['expected_output_column']  #@param {type:"string"}  # 正解 # expected_outputは，期待される出力をヘッダー名で指定します．
        )

        correctness_metric.measure(test_case)  # スコアを計算 # correctness_metricを使用して，テストケースのスコアを計算します．
        scores.append(correctness_metric.score)  # スコアをリストに追加 # 計算されたスコアをscoresリストに追加します．
        reasons.append(correctness_metric.reason)  # 理由をリストに追加 # スコアの理由をreasonsリストに追加します．
    except TimeoutException:
        print(f"Timeout occurred for row {row.name}")
        scores.append(None)  # タイムアウトが発生した場合，スコアをリストに追加
        reasons.append(None)  # タイムアウトが発生した場合，理由をリストに追加
    except Exception as e:
        if "Connection" in str(e):
            raise  # ConnectionErrorが発生したら実行を停止
        else:
            print(f"Error evaluating row {row.name}: {e}")
            correctness_scores.append(None)
            correctness_reasons.append(None)

# データフレームにスコアと理由を追加
data['Score'] = scores # データフレームに"Score"列を追加し，scoresリストの値を設定します．
data['Reason'] = reasons # データフレームに"Reason"列を追加し，reasonsリストの値を設定します．

# 修正後のデータフレームをTSVファイルとして出力
data.to_csv(output_file, sep='\t', index=False) # データフレームをTSVファイルとして出力します．sep='\t'は，タブ区切り値（TSV）形式でファイルを保存することを指定します．index=Falseは，インデックスを出力しないことを指定します．

print(f"Scores and reasons added. Output saved to {output_file}") # 出力ファイルのパスを表示します．

In [ ]:
#@title ランタイムの切断
from google.colab import runtime
runtime.unassign()

In [ ]:
@title tsvファイルを出力しない場合のコード
from deepeval.test_case import LLMTestCase
...

test_case = LLMTestCase(
input = "The dog chased the cat up the tree, who ran up the tree?", # @param {"type":"string"}
    # actual_output="It depends, some might consider the cat, while others might argue the dog.",
    actual_output="The cat .",  #@param
    expected_output="The cat."  #@param
)

correctness_metric.measure(test_case)
print(correctness_metric.score)
print(correctness_metric.reason)